### Baseline
rating = α ∗ μi + β ∗ ηj

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# import the data covert that into a convenient data
PATH= '/home/ac-optimus/data_science/recommendation_sys/ratings.csv'
df = pd.read_csv(PATH, sep= ",")[:100000]
# df = pd.read_csv(PATH, sep= ",")
df.shape

(100000, 4)

In [4]:
df.drop('timestamp', axis=1, inplace=True)
df.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [5]:
# this gave me a good plot
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
df.head()

,userId,movieId,rating
54208,567,4388,1.0
72599,751,1281,4.0
15690,193,1653,4.0
20815,236,1136,5.0
21044,242,78499,3.0


In [7]:
num_rows = df.shape[0]
train, test = df.copy(deep = True), df[int(num_rows*0.8):].copy(deep = True)
train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0
train = shuffle(train)

<ipython-input-7-75ff325d5135>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0


In [8]:
user_movie_rating =train.groupby(['userId','movieId'])['rating'].max().unstack().fillna(0)
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,173209,173291,173353,174055,174231,174393,174585,174843,175579,176271
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def error(y_hat, y):
    # MSE
    y_hat = np.array(y_hat)
    y = np.array(y)
    MSE_ = (y_hat- y)**2
    MSE = MSE_.sum()
    return MSE

In [10]:
# avg_user_rating(μi) - the average rating of user i
# avg_movie_rating(ηj) - that of movie j over all ratings  
# α and β are fitted using training data. I am using linear regression for this.

avg_user_rating = user_movie_rating.mean(axis=1).tolist()
avg_movie_rating = user_movie_rating.mean(axis=0).tolist()
len(avg_movie_rating), len(avg_user_rating)

(8832, 1014)

In [11]:
# mapping movieid, userid and corresponding average ratings.
columns = user_movie_rating.columns.tolist()
indexes = user_movie_rating.index.tolist()

new_df =  df.copy(deep=True)
new_df['movie_avg_rating'] = new_df['movieId'].apply(lambda x:avg_movie_rating[columns.index(x)])
new_df['user_movie_rating'] = new_df['userId'].apply(lambda x:avg_user_rating[indexes.index(x)])
new_df.head()

,userId,movieId,rating,movie_avg_rating,user_movie_rating
54208,567,4388,1.0,0.036982,0.248641
72599,751,1281,4.0,0.051282,0.094599
15690,193,1653,4.0,0.278107,0.104846
20815,236,1136,5.0,0.513314,0.070992
21044,242,78499,3.0,0.146943,0.053159


In [12]:
# baesline test and train datasets. It is different from the above test train test split. 
# Follows the usual train-test split
baseline_train = new_df[:int(num_rows*0.8)].drop(['userId', 'movieId'], axis=1)
baseline_test =  new_df[int(num_rows*0.8):].drop(['userId', 'movieId'], axis=1)
baseline_test.shape, baseline_train.shape

((20000, 3), (80000, 3))

In [13]:
from sklearn import datasets, linear_model

regr = linear_model.LinearRegression(fit_intercept= False)
regr.fit(baseline_train[['movie_avg_rating', 'user_movie_rating']], baseline_train[['rating']])

LinearRegression(fit_intercept=False)

In [14]:
y= test['rating'].tolist()
y_hat = regr.predict(baseline_test[['movie_avg_rating', 'user_movie_rating']])

# The coefficients
print('α: {}\nβ: {}'.format(regr.coef_[0][0], regr.coef_[0][1]))

α: 6.427386738381297
β: 7.659495184356608


In [15]:
error(y_hat, y)

2206842412.2937646

In [16]:
baseline_train

,rating,movie_avg_rating,user_movie_rating
54208,1.0,0.036982,0.248641
72599,4.0,0.051282,0.094599
15690,4.0,0.278107,0.104846
20815,5.0,0.513314,0.070992
21044,3.0,0.146943,0.053159
...,...,...,...
14046,4.0,0.095168,0.085145
48745,0.5,0.123767,0.052423
9153,2.0,0.212525,0.016304
42152,5.0,0.727811,0.046875


In [20]:
%%time
# looking for optimal k
y= test['rating']
errors = {}
y_hat1 = []
misses = 0
for i in test.index:
    user_i, movie_j = map(int,[test.loc[i]['userId'], 
                               test.loc[i]['movieId']])
    # make the prediction here
#     predicted_rating = PredictMovieRating(user_i, 
#                                           movie_j, 
#                                           correlation_matrix, 
#                                           user_movie_rating, k)
    if movie_j in baseline_train['movie_avg_rating']:
        predicted_rating = baseline_train['movie_avg_rating'][movie_j]
    else:
        misses+=1
        predicted_rating = 0
    y_hat1.append(predicted_rating)
error_i = error(np.nan_to_num(np.array(y_hat1)), y)        


CPU times: user 11.1 s, sys: 1.54 s, total: 12.6 s
Wall time: 12.7 s


In [22]:
error_i = error(np.nan_to_num(np.array(y_hat1)), y)        
error_i

249636.6961345405

In [23]:
misses

4708

In [ ]:
len(y), len(y_hat1), misses

In [ ]:
plt.plot([(i, j) for i, j in zip(y_hat1, y)])

In [ ]:
plt.plot([(i, j) for i, j in zip(y_hat, y)])

In [24]:
len(test), len(train)

(20000, 100000)